[View in Colaboratory](https://colab.research.google.com/github/FranciscoRencoret1/deeplearning/blob/master/AlexNet.ipynb)

In [3]:
!wget https://www.dropbox.com/s/c0d7npcg1yysnux/MIT-10-Classes.zip?dl=0
!unzip -q MIT-10-Classes.zip?dl=0

--2018-04-21 19:34:28--  https://www.dropbox.com/s/c0d7npcg1yysnux/MIT-10-Classes.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/L5S6Z1kwskaXIroZmvk3ElWSovtMspDVicM7d1Rdc0zQAleCbOAFD96dtiofctEw/file [following]
--2018-04-21 19:34:29--  https://dl.dropboxusercontent.com/content_link/L5S6Z1kwskaXIroZmvk3ElWSovtMspDVicM7d1Rdc0zQAleCbOAFD96dtiofctEw/file
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.7.6, 2620:100:6016:6::a27d:106
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.7.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 139459806 (133M) [application/zip]
Saving to: ‘MIT-10-Classes.zip?dl=0’

MIT-10-Classes.zip? 100%[===================>] 133.00M  33.7MB/s    in 4.0s    


In [0]:
#Definicion de librerias con la funciones que seran utilizadas por Keras.
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import to_categorical

In [6]:
#Definicion de contenedor y primera capa de AlexNet.
modelAlexNet = Sequential()
modelAlexNet.add(ZeroPadding2D((2,2), input_shape=(224, 224, 3)))
modelAlexNet.add(Convolution2D(96, (11,11), strides=(4,4), padding="valid"))
modelAlexNet.add(Activation(activation="relu"))
output_shape_first_layer = modelAlexNet.output_shape
print('Los mapas de activación de la capa 1 de convolción tienen tamaño: {}'.format(output_shape_first_layer))
modelAlexNet.add(BatchNormalization())
modelAlexNet.add(MaxPooling2D((3,3), strides=(2,2)))

Los mapas de activación de la capa 1 de convolción tienen tamaño: (None, 55, 55, 96)


In [7]:
#Definicion de la segunda capa de AlexNet.
modelAlexNet.add(ZeroPadding2D((2,2)))
modelAlexNet.add(Convolution2D(256, (5, 5), padding="valid"))
modelAlexNet.add(Activation(activation="relu"))
output_shape_second_layer = modelAlexNet.output_shape
print('Los mapas de activación de la capa 2 de convolción tienen tamaño: {}'.format(output_shape_second_layer))
modelAlexNet.add(BatchNormalization())
modelAlexNet.add(MaxPooling2D((3,3), strides=(2,2)))

Los mapas de activación de la capa 2 de convolción tienen tamaño: (None, 27, 27, 256)


In [8]:
#Definicion de la tercera capa de AlexNet
modelAlexNet.add(ZeroPadding2D((1,1)))
modelAlexNet.add(Convolution2D(384, (3, 3), strides=(1,1), padding="valid"))
modelAlexNet.add(Activation(activation="relu"))
output_shape_third_layer = modelAlexNet.output_shape
print('Los mapas de activación de la capa 3 de convolción tienen tamaño: {}'.format(output_shape_third_layer))

Los mapas de activación de la capa 3 de convolción tienen tamaño: (None, 13, 13, 384)


In [9]:
#Definicion de la cuarta capa de AlexNet
modelAlexNet.add(ZeroPadding2D((1,1)))
modelAlexNet.add(Convolution2D(384, (3, 3), strides=(1,1), padding="valid"))
modelAlexNet.add(Activation(activation="relu"))
output_shape_fourth_layer = modelAlexNet.output_shape
print('Los mapas de activación de la capa 4 de convolción tienen tamaño: {}'.format(output_shape_fourth_layer))

Los mapas de activación de la capa 4 de convolción tienen tamaño: (None, 13, 13, 384)


In [10]:
#Definicion de la quinta capa de AlexNet
modelAlexNet.add(ZeroPadding2D((1,1)))
modelAlexNet.add(Convolution2D(256, (3, 3), strides=(1,1), padding="valid"))
modelAlexNet.add(Activation(activation="relu"))
output_shape_fifth_layer = modelAlexNet.output_shape
print('Los mapas de activación de la capa 5 de convolción tienen tamaño: {}'.format(output_shape_fifth_layer))
modelAlexNet.add(MaxPooling2D((3,3), strides=(2,2)))

Los mapas de activación de la capa 5 de convolción tienen tamaño: (None, 13, 13, 256)


In [0]:
modelAlexNet.add(Flatten())

In [0]:
#Definicion de la capa 6 de AlexNet
modelAlexNet.add(Dense(4096, input_shape=(9216,), activation='relu', kernel_initializer='glorot_uniform', use_bias=True))
modelAlexNet.add(Dropout(0.5))
output_shape_sixth_layer = modelAlexNet.output_shape

In [0]:
#Definición de la capa 7 de AlexNet
modelAlexNet.add(Dense(4096, activation='relu', kernel_initializer='glorot_uniform', use_bias=True))
modelAlexNet.add(Dropout(0.5))
output_shape_seventh_layer = modelAlexNet.output_shape

In [0]:
#Definición de la capa 8 de AlexNet
modelAlexNet.add(Dense(10, activation='softmax', kernel_initializer='glorot_uniform', use_bias=True))
output_shape_eighths_layer = modelAlexNet.output_shape

In [0]:
modelAlexNet.compile(optimizer=optimizers.Adam(lr=5e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
#Load the images
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './MIT-10-Classes/Imgs/TrainSetImgs',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')


Found 799 images belonging to 10 classes.
Found 201 images belonging to 10 classes.


In [17]:
modelAlexNet.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.samples / 32,
        epochs=25,
        verbose=1)

Epoch 1/50
25/24 [==============================] - 9s 340ms/step - loss: 3.0145 - acc: 0.1100
Epoch 2/50
25/24 [==============================] - 7s 275ms/step - loss: 2.1548 - acc: 0.2179
Epoch 3/50
25/24 [==============================] - 7s 274ms/step - loss: 2.0155 - acc: 0.2792
Epoch 4/50
25/24 [==============================] - 7s 272ms/step - loss: 1.9025 - acc: 0.3328
Epoch 5/50
25/24 [==============================] - 7s 275ms/step - loss: 1.7763 - acc: 0.3729
Epoch 6/50
25/24 [==============================] - 7s 274ms/step - loss: 1.6151 - acc: 0.4105
Epoch 7/50
25/24 [==============================] - 7s 274ms/step - loss: 1.5132 - acc: 0.4705
Epoch 8/50
25/24 [==============================] - 7s 274ms/step - loss: 1.4552 - acc: 0.5005
Epoch 9/50
14/24 [===============>..............] - ETA: 3s - loss: 1.2674 - acc: 0.5861

25/24 [==============================] - 7s 273ms/step - loss: 1.2696 - acc: 0.5795
Epoch 10/50
25/24 [==============================] - 7s 273ms/step - loss: 1.1245 - acc: 0.6144
Epoch 11/50
25/24 [==============================] - 6s 255ms/step - loss: 0.8619 - acc: 0.6971
Epoch 12/50
25/24 [==============================] - 7s 277ms/step - loss: 0.7394 - acc: 0.7648
Epoch 13/50
25/24 [==============================] - 7s 273ms/step - loss: 0.7013 - acc: 0.7748
Epoch 14/50
25/24 [==============================] - 7s 273ms/step - loss: 0.5136 - acc: 0.8223
Epoch 15/50
25/24 [==============================] - 7s 271ms/step - loss: 0.4315 - acc: 0.8623
Epoch 16/50
25/24 [==============================] - 7s 272ms/step - loss: 0.4027 - acc: 0.8610
Epoch 17/50
16/24 [==================>...........] - ETA: 2s - loss: 0.3003 - acc: 0.9160

25/24 [==============================] - 7s 273ms/step - loss: 0.2992 - acc: 0.9098
Epoch 18/50
25/24 [==============================] - 7s 274ms/step - loss: 0.2304 - acc: 0.9249
Epoch 19/50
25/24 [==============================] - 7s 273ms/step - loss: 0.2728 - acc: 0.9134
Epoch 20/50
25/24 [==============================] - 7s 272ms/step - loss: 0.4047 - acc: 0.8922
Epoch 21/50
25/24 [==============================] - 7s 274ms/step - loss: 0.3157 - acc: 0.9012
Epoch 22/50
25/24 [==============================] - 7s 273ms/step - loss: 0.2021 - acc: 0.9274
Epoch 23/50
25/24 [==============================] - 7s 274ms/step - loss: 0.1527 - acc: 0.9499
Epoch 24/50
25/24 [==============================] - 7s 274ms/step - loss: 0.1705 - acc: 0.9475
Epoch 25/50
17/24 [===================>..........] - ETA: 2s - loss: 0.2273 - acc: 0.9337

25/24 [==============================] - 7s 276ms/step - loss: 0.2240 - acc: 0.9312
Epoch 26/50
25/24 [==============================] - 7s 274ms/step - loss: 0.2176 - acc: 0.9448
Epoch 27/50
25/24 [==============================] - 7s 273ms/step - loss: 0.2533 - acc: 0.9337
Epoch 28/50
25/24 [==============================] - 7s 271ms/step - loss: 0.1786 - acc: 0.9474
Epoch 29/50
25/24 [==============================] - 7s 276ms/step - loss: 0.1251 - acc: 0.9637
Epoch 30/50
25/24 [==============================] - 7s 272ms/step - loss: 0.1669 - acc: 0.9500
Epoch 31/50
25/24 [==============================] - 7s 272ms/step - loss: 0.1519 - acc: 0.9500
Epoch 32/50
25/24 [==============================] - 7s 273ms/step - loss: 0.0859 - acc: 0.9737
Epoch 33/50
16/24 [==================>...........] - ETA: 2s - loss: 0.0783 - acc: 0.9746

25/24 [==============================] - 7s 273ms/step - loss: 0.0705 - acc: 0.9787
Epoch 34/50
25/24 [==============================] - 7s 274ms/step - loss: 0.1080 - acc: 0.9687
Epoch 35/50
25/24 [==============================] - 7s 275ms/step - loss: 0.2308 - acc: 0.9450
Epoch 36/50
25/24 [==============================] - 7s 272ms/step - loss: 0.2349 - acc: 0.9412
Epoch 37/50
25/24 [==============================] - 7s 271ms/step - loss: 0.1962 - acc: 0.9412
Epoch 38/50
25/24 [==============================] - 7s 273ms/step - loss: 0.2061 - acc: 0.9448
Epoch 39/50
25/24 [==============================] - 7s 268ms/step - loss: 0.1793 - acc: 0.9487
Epoch 40/50
25/24 [==============================] - 7s 270ms/step - loss: 0.0557 - acc: 0.9812
Epoch 41/50
16/24 [==================>...........] - ETA: 2s - loss: 0.1019 - acc: 0.9726

25/24 [==============================] - 7s 272ms/step - loss: 0.0999 - acc: 0.9700
Epoch 42/50
25/24 [==============================] - 7s 271ms/step - loss: 0.1116 - acc: 0.9700
Epoch 43/50
25/24 [==============================] - 7s 271ms/step - loss: 0.1471 - acc: 0.9687
Epoch 44/50
25/24 [==============================] - 7s 276ms/step - loss: 0.2206 - acc: 0.9411
Epoch 45/50
25/24 [==============================] - 7s 273ms/step - loss: 0.2958 - acc: 0.9387
Epoch 46/50
25/24 [==============================] - 7s 280ms/step - loss: 0.1771 - acc: 0.9486
Epoch 47/50
25/24 [==============================] - 7s 278ms/step - loss: 0.1990 - acc: 0.9575
Epoch 48/50
25/24 [==============================] - 7s 278ms/step - loss: 0.1885 - acc: 0.9625
Epoch 49/50
17/24 [===================>..........] - ETA: 2s - loss: 0.1912 - acc: 0.9593

25/24 [==============================] - 7s 277ms/step - loss: 0.1554 - acc: 0.9636
Epoch 50/50
25/24 [==============================] - 7s 280ms/step - loss: 0.1021 - acc: 0.9661


In [18]:
print("Saving Model")
modelAlexNet.save('modelAlexNet.h5')

Saving Model


In [0]:
import scipy.io as sio
import numpy as np
import cv2
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [0]:
def load_test_set(directory):
    #Load the clases
    cls = os.listdir(directory)
    #Create empty arrays
    X = []
    Y = []
    #Load the set
    for cl in cls:
        for img in os.listdir(directory + '/' + cl):
            img_arr = img_to_array(load_img(directory + '/' + cl + '/' + img))
            resized = cv2.resize(img_arr, (224, 224))
            X.append(resized)
            Y.append(cl)
    return np.array(X), np.array(Y)

X_test, Y_test_string = load_test_set('./MIT-10-Classes/Imgs/TestSetImgs')

In [0]:
# Convert string classes to vector of dim (1,10). Vector has a 1 on the
# corresponding class and 0 on the rest.
encoder = LabelEncoder()
encoder.fit(Y_test_string)
encoded_Y = encoder.transform(Y_test_string)
Y_test = to_categorical(encoded_Y)

In [96]:
Y_pred = modelAlexNet.predict(X_test)
res = 0
for i in range(Y_pred.shape[0]):
  if np.array_equal(Y_pred[i], Y_test[i]) == True:
    res += 1
print(res)
print(res/Y_pred.shape[0])

40
0.19900497512437812
